In [1]:
import datetime
import requests
import collections
import json
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def getAccessToken():
    client_id = 'e088d78512324711aab5a7d2c5c7945c'
    client_secret = 'b65c9cd8db494a8faa7be0535f762b55'

    grant_type = 'client_credentials'

    #Request based on Client Credentials Flow from https://developer.spotify.com/web-api/authorization-guide/  
    #Request body parameter: grant_type Value: Required. We use client_credentials to get an access token for carrying out requests.
    body_params = {'grant_type' : grant_type}

    url='https://accounts.spotify.com/api/token'

    response=requests.post(url, data=body_params, auth = (client_id, client_secret)) 
    response=(json.loads(response.text))

    accessToken=(response['access_token'])

    return accessToken

In [3]:
fromDate=datetime.date(2016,12,23)
toDate=fromDate + datetime.timedelta(weeks=1)
fromDate=fromDate.strftime('%Y-%m-%d')
toDate=toDate.strftime('%Y-%m-%d')
df= pd.read_csv('Data/regional-us-weekly-'+fromDate+'--'+toDate+'.csv')
df['month']=datetime.datetime.strptime(fromDate,'%Y-%m-%d').strftime('%B')
df.head()

,Position,Track Name,Artist,Streams,URL,month
0,1,Fake Love,Drake,8618267,https://open.spotify.com/track/343YBumqHu19cGo...,December
1,2,Bad and Boujee (feat. Lil Uzi Vert),Migos,7946032,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,December
2,3,Starboy,The Weeknd,7738517,https://open.spotify.com/track/5aAx2yezTd8zXrk...,December
3,4,Black Beatles,Rae Sremmurd,6209336,https://open.spotify.com/track/6fujklziTHa8uoM...,December
4,5,Closer,The Chainsmokers,5813157,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,December


In [23]:
def trackId(trackLink):
    return trackLink.split('/')[4]
df['trackId']=df['URL'].apply(trackId)
df.head()

,Position,Track Name,Artist,Streams,URL,month,trackId,Danceability,Energy,Key,Loudness,Tonality,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,TimeSignature
0,1,Fake Love,Drake,8618267,https://open.spotify.com/track/343YBumqHu19cGo...,December,343YBumqHu19cGoGARUTsd,0.853,0.656,7,-7.011,0,0.0816,0.0403,0.000006,0.0889,0.857,113.991,4
1,2,Bad and Boujee (feat. Lil Uzi Vert),Migos,7946032,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,December,4Km5HrUvYTaSUfiSGPJeQR,0.853,0.656,7,-7.011,0,0.0816,0.0403,0.000006,0.0889,0.857,113.991,4
2,3,Starboy,The Weeknd,7738517,https://open.spotify.com/track/5aAx2yezTd8zXrk...,December,5aAx2yezTd8zXrkmtKl66Z,0.853,0.656,7,-7.011,0,0.0816,0.0403,0.000006,0.0889,0.857,113.991,4
3,4,Black Beatles,Rae Sremmurd,6209336,https://open.spotify.com/track/6fujklziTHa8uoM...,December,6fujklziTHa8uoM5OQSfIo,0.853,0.656,7,-7.011,0,0.0816,0.0403,0.000006,0.0889,0.857,113.991,4
4,5,Closer,The Chainsmokers,5813157,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,December,7BKLCZ1jbUBVqRi2FVlTVw,0.853,0.656,7,-7.011,0,0.0816,0.0403,0.000006,0.0889,0.857,113.991,4


In [27]:
# df.drop(df.columns[7:],axis=1,inplace=True)
df.head()

,Position,Track Name,Artist,Streams,URL,month,trackId
0,1,Fake Love,Drake,8618267,https://open.spotify.com/track/343YBumqHu19cGo...,December,343YBumqHu19cGoGARUTsd
1,2,Bad and Boujee (feat. Lil Uzi Vert),Migos,7946032,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,December,4Km5HrUvYTaSUfiSGPJeQR
2,3,Starboy,The Weeknd,7738517,https://open.spotify.com/track/5aAx2yezTd8zXrk...,December,5aAx2yezTd8zXrkmtKl66Z
3,4,Black Beatles,Rae Sremmurd,6209336,https://open.spotify.com/track/6fujklziTHa8uoM...,December,6fujklziTHa8uoM5OQSfIo
4,5,Closer,The Chainsmokers,5813157,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,December,7BKLCZ1jbUBVqRi2FVlTVw


In [32]:
def getAudioFeatures(songId):
    accessToken=getAccessToken()
    headers = {'Authorization' : 'Bearer '+accessToken}
    url='https://api.spotify.com/v1/audio-features/'+songId
    response=requests.get(url, headers=headers) 
    response=(json.loads(response.text))
    return response
#     df['Danceability']=response['danceability']
#     df['Energy']=response['energy']
#     df['Key']=response['key']
#     df['Loudness']=response['loudness']
#     df['Tonality']=response['mode']
#     df['Speechiness']=response['speechiness']
#     df['Acousticness']=response['acousticness']
#     df['Instrumentalness']=response['instrumentalness']
#     df['Liveness']=response['liveness']
#     df['Valence']=response['valence']
#     df['Tempo']=response['tempo']
#     df['TimeSignature']=response['time_signature']

IndexError: invalid index to scalar variable.